# 🚀 Emotion Classification with BERT-based Models

This notebook reproduces the results for **Project 2: Emotion Classification**.
It covers setup, downloading pre-trained weights, training, evaluation, and compression benchmarks.

### ⚠️ Hardware Requirement: GPU Required
**Please ensure you are connected to a GPU Runtime.**
- Go to `Runtime` > `Change runtime type` > `Hardware accelerator` > Select **T4 GPU** (or A100/V100).
- **Do NOT use TPU**: The compression library `bitsandbytes` (used for 4-bit quantization) currently relies on CUDA kernels and is not compatible with Colab TPUs.

## 1. Setup & Installation

In [ ]:
# 1. Clone the repository
!git clone https://github.com/Roeyx/NLP.git

# 2. Enter the project directory
%cd NLP/Project2

# 3. Install dependencies
!pip install -r requirements.txt

# 4. Create outputs directory
!mkdir -p outputs

## 2. Download Fine-Tuned Weights
We download the checkpoints that were fine-tuned on the Emotion Classification task to skip the training phase.

In [ ]:
# === 1. Best CardiffRoBERTa (Original FP32) ===
!gdown 1hU4AA1mIE6fJHYEYUB0lgwBnyiESjoFK -O outputs/best_CardiffRoBERTa.pt

# === 2. CardiffRoBERTa Variants (Compressed) ===
# Pruned (30%)
!gdown 1_ZjdBnr08MZRZZ6QkIikwVhXb3FH7xgH -O outputs/best_CardiffRoBERTa_pruned_30.pt
# Quantized INT8
!gdown 1tRtZGKaADsVQ3_5APiYQXFwK4IjB4mq9 -O outputs/best_CardiffRoBERTa_quantized_int8.pt
# Quantized NF4
!gdown 1bYhpRRKBpJu3RNcOG753ss4SMXMeqwPA -O outputs/best_CardiffRoBERTa_quantized_nf4.pt

# === 3. Other Models (Fine-tuned FP32) ===
# ModernBERT
!gdown 1T3U4lDa9_MMG2vbrPqkvXTWFWfYXfF7Z -O outputs/best_ModernBERT.pt
# BERTweet
!gdown 1UfmmuODYVqRkp138kZ8Ya7WknO8jiGqu -O outputs/best_BERTweet.pt

## 3. Training Demo
Train a fresh model from scratch. Uncomment specific models to run them.

In [ ]:
# Train ModernBERT (Demo - Active)
# !python emotion_classifier.py --mode train --model ModernBERT

# Train CardiffRoBERTa
# !python emotion_classifier.py --mode train --model CardiffRoBERTa

# Train BERTweet
# !python emotion_classifier.py --mode train --model BERTweet

# Train ALL Models
# !python emotion_classifier.py --mode train

## 4. Evaluation (Reproduce Results)
Evaluate the downloaded models on the validation set to verify Accuracy and F1 scores.

In [ ]:
print("=== Evaluating Original CardiffRoBERTa (FP32) ===")
!python emotion_classifier.py --mode evaluate --weights outputs/best_CardiffRoBERTa.pt --val validation.csv

print("\n=== Evaluating Pruned CardiffRoBERTa (30%) ===")
!python emotion_classifier.py --mode evaluate --weights outputs/best_CardiffRoBERTa_pruned_30.pt --val validation.csv

print("\n=== Evaluating Quantized CardiffRoBERTa (INT8) ===")
!python emotion_classifier.py --mode evaluate --weights outputs/best_CardiffRoBERTa_quantized_int8.pt --val validation.csv

print("\n=== Evaluating Quantized CardiffRoBERTa (NF4) ===")
!python emotion_classifier.py --mode evaluate --weights outputs/best_CardiffRoBERTa_quantized_nf4.pt --val validation.csv

In [ ]:
print("=== Evaluating ModernBERT ===")
!python emotion_classifier.py --mode evaluate --weights outputs/best_ModernBERT.pt --val validation.csv

print("\n=== Evaluating BERTweet ===")
!python emotion_classifier.py --mode evaluate --weights outputs/best_BERTweet.pt --val validation.csv

## 5. Inference on Custom Test Data
Upload your own `test.csv` file (must contain a 'text' column) to generate predictions.

In [ ]:
from google.colab import files
import os

print("Please upload your test.csv file:")
uploaded = files.upload()

filename = list(uploaded.keys())[0]
print(f"Uploaded: {filename}")

In [ ]:
# Run inference using CardiffRoBERTa
output_roberta = f"{os.path.splitext(filename)[0]}_predictions_roberta.csv"
print(f"\n=== Inference: CardiffRoBERTa -> {output_roberta} ===")
!python emotion_classifier.py --mode inference --weights outputs/best_CardiffRoBERTa.pt --test_csv "{filename}"
!mv "{os.path.splitext(filename)[0]}_predictions.csv" "{output_roberta}"

# Run inference using ModernBERT
output_modern = f"{os.path.splitext(filename)[0]}_predictions_modern.csv"
print(f"\n=== Inference: ModernBERT -> {output_modern} ===")
!python emotion_classifier.py --mode inference --weights outputs/best_ModernBERT.pt --test_csv "{filename}"
!mv "{os.path.splitext(filename)[0]}_predictions.csv" "{output_modern}"

# Run inference using BERTweet
output_bertweet = f"{os.path.splitext(filename)[0]}_predictions_bertweet.csv"
print(f"\n=== Inference: BERTweet -> {output_bertweet} ===")
!python emotion_classifier.py --mode inference --weights outputs/best_BERTweet.pt --test_csv "{filename}"
!mv "{os.path.splitext(filename)[0]}_predictions.csv" "{output_bertweet}"

# Download the results (Defaulting to RoBERTa)
print("\nDownloading RoBERTa predictions...")
files.download(output_roberta)

## 6. Compression Analysis
Run the full compression benchmark suite (Pruning, INT8, NF4, Combined) to generate the comparison report.

**Note:** This step requires the GPU runtime for `bitsandbytes` quantization.

In [ ]:
# Benchmark CardiffRoBERTa (Active)
!python emotion_classifier.py --mode compress --weights outputs/best_CardiffRoBERTa.pt --val validation.csv

# Benchmark ModernBERT
# !python emotion_classifier.py --mode compress --weights outputs/best_ModernBERT.pt --val validation.csv

# Benchmark BERTweet
# !python emotion_classifier.py --mode compress --weights outputs/best_BERTweet.pt --val validation.csv